In [1]:
from sqlalchemy import types
from StringIO import StringIO
import sqlite3
import CleanData as cd
import numpy as np
import pandas as pd

# Clean Data

In [2]:
title = 'Roman_Imperial_Coinage'

### British Museum

In [3]:
# Read in data and convert to appropriate types
bm_df = pd.read_csv('../Data/BM_'+title+'.csv',
                 converters={"Authority": cd.stringToList(), 'Associated names': cd.stringToList(), 
                             'Subjects': cd.stringToList(), 'Inscriptions': cd.stringToListofDicts('|', ';', ':')})
bm_df = pd.concat([bm_df, cd.cleanInscriptions(bm_df)], axis=1)
bm_df = bm_df.drop('Unnamed: 0', 1)
bm_df = bm_df.replace(np.nan, '', regex=True)
bm_df = bm_df.rename(columns = {'Culture/period':'Culture'})

In [4]:
# Clean data to only retain relevant information
mask = ((bm_df['Object type'] == 'coin ') & (bm_df['Date'].str.find('stC') == -1))
filtered = bm_df[mask]

lists = ['Authority', 'Subjects', 'Associated names', 'Obverse legend', 'Reverse legend']
strings = ['Museum number', 'Denomination', 'Description', 'State', 'Culture', 'Materials', 
            'Curator\'s comments', 'Bibliography', 'Object type', "Production place"]
floats = ['Weight (g)']
dates = ['Date']
redundant_notes = ['Production place', 'Denomination']
do_nothing = ['url', 'Inscriptions']
duplicate_cols = ['Authority', 'Date', 'Production place', 'Description', 'Subjects', "Curator's comments",
                  'Obverse legend', 'Reverse legend']

cleaned_bm = cd.cleanDF(filtered, lists, strings, floats, dates, redundant_notes, do_nothing, duplicate_cols)
cleaned_bm = cleaned_bm.drop('Inscriptions', 1)
cleaned_bm['Source'] = 'British Museum'
#cleaned_bm.head()

In [5]:
# Convert everything to strings and integers to put into SQL db
lists = ['Authority', 'Subjects', 'Associated names', 'Obverse legend', 'Reverse legend', 'Date']
for col in lists:
    cleaned_bm[col] = cleaned_bm[col].apply(cd.listToString())

columns = [u'Associated names', u'Authority', u'Bibliography', u'Culture',
       u'Curator\'s comments', u'Date', u'Denomination', u'Description',
       u'Materials', u'Museum number', u'Object type', u'Obverse legend',
       u'Production place', u'Reverse legend', u'State', u'Subjects',
       u'url', u'Source']

for col in columns:
    cleaned_bm[col] = cleaned_bm[col].apply(lambda x: x.decode('utf-8'))

cleaned_bm.columns = ['associatedNames', 'authority', 'bibliography', 'culture',
                       'curatorComment', 'date', 'denomination', 'description',
                       'materials', 'museumNumber', 'objectType', 'obverseLegend',
                       'mint', 'reverseLegend', 'state', 'subjects',
                       'weight', 'url', 'source']

cleaned_bm['startDate'] = cleaned_bm.apply(lambda row: row['date'].split(",")[0], axis=1)
cleaned_bm['endDate'] = cleaned_bm.apply(lambda row: row['date'].split(",")[1] 
                                                 if len(row['date'].split(",")) > 1 
                                                 else row['date'].split(",")[0], 
                                         axis=1)

In [6]:
cleaned_bm.tail()

,associatedNames,authority,bibliography,culture,curatorComment,date,denomination,description,materials,museumNumber,...,obverseLegend,mint,reverseLegend,state,subjects,weight,url,source,startDate,endDate
22024,"Antinous, Demeter/Ceres",Hadrian,"BMC Greek (Mysia) 13, p.4 PCG 86.3, p. (pl. 47...",Roman Provincial,The young courtier Antinous died suddenly in A...,"130, 138",?,"Alloy coin.(obverse) Bare head of Antinous, ri...",alloy,"1844,0425.1609",...,AHTINOOC IAKXOC,Adramyteum,ECIOC AN HKE A PAMVTHNOIC,Roman Empire,classical deity,26.69,http://www.britishmuseum.org/research/collecti...,British Museum,130,138
22025,Constantine the Great,Constantine the Great,"RIC6 65, p.407 (type)",Roman Imperial,,"312, 313",solidus,"Gold coin.(obverse) Bust of Constantine I, dra...",gold,"1896,0608.98",...,IMP CONSTANTINVS P F AVG,Ostia,PRINCIPI IVVENTVTIS,Roman Empire,emperor/empress,8.49,http://www.britishmuseum.org/research/collecti...,British Museum,312,313
22026,Hadrian,Hadrian,"RE3 1209, p. 417 Strack (Hadrian) 557 (var = f...",Roman Imperial,,"117, 138",sestertius,"on the left, three citizens in front and one b...",copper alloy,"1869,0507.7",...,IMP CAESAR TRAIANVS HADRIANVS AVG P M TR P COS...,Rome,RELIQVA VETERA HS N[OVIES] MILL ABOLITA,Roman Empire,"politics, emperor/empress",24.07,http://www.britishmuseum.org/research/collecti...,British Museum,117,138
22027,Hadrian,Hadrian,"RE3 1207, p. 417 Strack (Hadrian) 557 RIC2 592...",Roman Imperial,,"117, 138",sestertius,"on the left, three citizens (two in front, one...",copper alloy,"1872,0709.561",...,IMP CAESAR TRAIANVS HADRIANVS AVG P M TR P COS...,Rome,RELIQVA VETERA HS NOVIES MILL ABOLITA,Roman Empire,"politics, emperor/empress",20.64,http://www.britishmuseum.org/research/collecti...,British Museum,117,138
22028,Constantine the Great,Constantine the Great,RIC7 p275.381,Roman Imperial,maiorina (big [coins]) and centenionalis (a ‘h...,334,nummus,"Copper alloy coin.(obverse) Diademed, draped a...",copper alloy,B.1746,...,CONSTANTINVS MAX AVG,Arles,GLORIA EXERCITVS,Roman Empire,"soldier, emperor/empress",2.28,http://www.britishmuseum.org/research/collecti...,British Museum,334,334


### American Numismatic Society

In [49]:
ans_df = pd.read_csv('../Data/ANS_'+title+'.csv',
                    converters={"Authority": cd.stringToList('|'), 'Year': cd.stringToList('|'),
                               'Issuer': cd.stringToList('|'), 'Portrait': cd.stringToList('|'),
                               'Reference': cd.stringToList('|')})
ans_df = ans_df.replace(np.nan, '', regex=True)
ans_df['Description'] = ans_df.apply(lambda row: cd.makeDescription(
                                                                    row['Material'], 
                                                                    row['Manufacture'], 
                                                                    row['Obverse Type'], 
                                                                    row['Reverse Type']
                                                                   ), axis=1)
ans_df['DupCheck'] = ans_df.apply(lambda row: cd.makeDupCheckCol(
                                                                 row['Material'], 
                                                                 row['Denomination'], 
                                                                 row['Portrait'], 
                                                                 row['Mint'],
                                                                 row['Year']
                                                                ), axis=1)
ans_df.tail()

,Unnamed: 0,URI,Title,RecordId,Authority,Coin Type URI,Date on Object,Degree,Deity,Denomination,...,Region,Reverse Legend,Reverse Type,Weight,Year,Thumbnail_obv,Thumbnail_rev,Date Record Modified,Description,DupCheck
62310,62310,http://numismatics.org/collection/2017.11.7,"Billon tetradrachm, Alexandreia, AD 118 - AD 1...",2017.11.7,[Hadrian],,,,Tyche,tetradrachm,...,Egypt,L Γ,Tyche standing l. holding rudder and cornucopia.,12.73,"[118, 119]",http://numismatics.org/collectionimages/200020...,http://numismatics.org/collectionimages/200020...,2017-12-05T17:40:30Z,Billon. (obverse) Laureate bust r. fold of cl...,"Billon tetradrachm [''] Alexandreia ['118', '1..."
62311,62311,http://numismatics.org/collection/2017.11.8,"Billon Tetradrachm, AD 122 - AD 123. 2017.11.8",2017.11.8,[Hadrian],,,,Athena|Nike,Tetradrachm,...,,L ϛ,Athena standing l. holding Nikeand shield,12.71,"[122, 123]",http://numismatics.org/collectionimages/200020...,http://numismatics.org/collectionimages/200020...,2017-12-05T17:40:30Z,"Billon. (obverse) Laureate bust r., fold of c...","Billon Tetradrachm [''] ['122', '123']"
62312,62312,http://numismatics.org/collection/2017.11.9,"Bronze Coin, Alexandreia, AD 164 - AD 165. 201...",2017.11.9,"[Faustina II under Marcus Aurelius,]",,,,,,...,Egypt,L E,"Eagle with wings folded standing left, head r.",7.8,"[164, 165]",,,2017-12-05T17:40:30Z,Bronze. (obverse) Bust draped r.. (reverse) E...,"Bronze [''] Alexandreia ['164', '165']"
62313,62313,http://numismatics.org/collection/2015.8.48,"Bronze AE3, Constantinople, AD 378 - AD 383. 2...",2015.8.48,[Theodosius I],,,,,AE3,...,,,,2.502,"[378, 383]",,,2017-12-19T12:57:47Z,Bronze.,"Bronze AE3 [''] Constantinople ['378', '383']"
62314,62314,http://numismatics.org/collection/2017.34.1,"Gold Aureus of Antoninus Pius, Rome, AD 145 - ...",2017.34.1,[Antoninus Pius],http://numismatics.org/ocre/id/ric.3.ant.503Aa,,,,Aureus,...,Italy,CONCORDIA,Dove standing right,7.11,"[145, 161]",http://numismatics.org/collectionimages/200020...,http://numismatics.org/collectionimages/200020...,2017-12-19T12:57:47Z,"Struck Gold. (obverse) Bust of Faustina II, dr...",Gold Aureus ['Faustina the Younger'] Rome ['14...


In [50]:
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
strings = ['Deity', 'Denomination', 'Mint', 'Description', 'Obverse Legend', 'Reverse Legend']
floats = ['Weight']
dates = []
redundant_notes = []
do_nothing = ['URI', 'DupCheck']
duplicate_cols = 'DupCheck'

cleaned_ans = cd.cleanDF(ans_df, lists, strings, floats, dates, redundant_notes, do_nothing, 
                              duplicate_cols, production_place='Mint')
cleaned_ans['Source'] = 'American Numismatic Society'
cleaned_ans.drop(['DupCheck'], axis=1, inplace=True)
cleaned_ans.tail()
cleaned_ans.columns

Index([u'Authority', u'Deity', u'Denomination', u'Description', u'Issuer',
       u'Mint', u'Obverse Legend', u'Portrait', u'Reference',
       u'Reverse Legend', u'URI', u'Weight', u'Year', u'Source'],
      dtype='object')

In [51]:
# Convert everything to strings and integers to put into SQL db
lists = ['Year', 'Authority', 'Issuer', 'Portrait', 'Reference']
for col in lists:
    cleaned_ans[col] = cleaned_ans[col].apply(cd.listToString())
    
cleaned_ans['StartDate'] = cleaned_ans.apply(lambda row: row['Year'].split(",")[0], axis=1)
cleaned_ans['EndDate'] = cleaned_ans.apply(lambda row: row['Year'].split(",")[1] 
                                                 if len(row['Year'].split(",")) > 1 
                                                 else row['Year'].split(",")[0], 
                                         axis=1)

columns = [u'Authority', u'Deity', u'Denomination', u'Description', u'Issuer',
       u'Mint', u'Obverse Legend', u'Portrait', u'Reference',
       u'Reverse Legend', u'URI', u'Weight', u'Year', u'Source', 'StartDate', 'EndDate']

cleaned_ans.columns = columns

for col in columns:
    cleaned_ans[col] = cleaned_ans[col].apply(lambda x: x.decode('utf-8') if type(x) is not float else x)

In [52]:
cleaned_ans.tail()

,Authority,Deity,Denomination,Description,Issuer,Mint,Obverse Legend,Portrait,Reference,Reverse Legend,URI,Weight,Year,Source,StartDate,EndDate
7826,Saloninus as Caesar,,tetradrachm,"Billon. (obverse) Bare-headed, draped, cuirass...",,Alexandreia,ΠΟΛΙΚΚΟΡCA OVAΛΕΡΙΑΝΟC K CEB,,"BMC 2296, Cologne 3002, Dattari 5376",L ϛ,http://numismatics.org/collection/2017.11.52,9.77,"258, 259",American Numismatic Society,258,259
7827,Claudius II,Tyche,tetradrachm,"Billon. (obverse) Bust laureate, draped, cuira...",,Alexandreia,ΑVTK KΛΑVΔΙΟC CEB,,"Cologne 3048 (same rev. die, Dattari-Savio pl....",L Γ,http://numismatics.org/collection/2017.11.53,9.69,"269, 270",American Numismatic Society,269,270
7828,Hadrian,Zeus,?,Billon. (obverse) Laureate bust r. fold of clo...,,,AYT KAIC TPAINOC (sic) ΑΔΡΙΑΝΟC,,RPC 5015 (2 spec.),L B,http://numismatics.org/collection/2017.11.6,13.74,"117, 118",American Numismatic Society,117,118
7829,Hadrian,Athena|Nike,Tetradrachm,"Billon. (obverse) Laureate bust r., fold of cl...",,,AYT KAI TRAI - AΔΡΙΑ CεΒ,,"Dattari-Savio, pl. 64, 7382, RPC.- cf. RPC 531...",L ϛ,http://numismatics.org/collection/2017.11.8,12.71,"122, 123",American Numismatic Society,122,123
7830,"Faustina II under Marcus Aurelius,",,?,Bronze. (obverse) Bust draped r.. (reverse) Ea...,,Alexandreia,ΦΑΥCTINA CεΒΑCΤΗ,,"Dattari-Savio pl. 195, 3639 (this coin)., Emme...",L E,http://numismatics.org/collection/2017.11.9,7.80,"164, 165",American Numismatic Society,164,165


# Store Data

In [11]:
cnx = sqlite3.connect('../Data/'+title+'.sqlite')
cur = cnx.cursor()

In [12]:
cur.execute('''CREATE TABLE britishMuseum(associatedNames TEXT, authority TEXT, bibliography TEXT, culture TEXT,
                    curatorComment TEXT, dates TEXT, denomination TEXT, 
                    description TEXT, materials TEXT, museumNumber TEXT, objectType TEXT, obverseLegend TEXT,
                    mint TEXT, reverseLegend TEXT, state TEXT, subjects TEXT, weight INT,
                    url TEXT, source TEXT, startDate INT, endDate INT)''')

In [13]:
cleaned_bm.loc[1]

associatedNames                                                     
authority                                                 Magnentius
bibliography                                                        
culture                                               Roman Imperial
curatorComment                                                      
date                                                        350, 353
denomination                                                       ?
description                                             Silver coin.
materials                                                     silver
museumNumber                                             1981,0745.6
objectType                                                      coin
obverseLegend                                                       
mint                                                        Aquileia
reverseLegend                                                       
state                             

In [14]:
list(cleaned_bm.to_records(index=False))[1]

(u'', u'Magnentius', u'', u'Roman Imperial', u'', u'350, 353', u'?', u'Silver coin.', u'silver', u'1981,0745.6', u'coin', u'', u'Aquileia', u'', u'Roman Empire', u'', 2.78, u'http://www.britishmuseum.org/research/collection_online/collection_object_details.aspx?objectId=2880&partId=1&searchText=roman+imperial+coins&lookup-people=e.g.+Hokusai%2c+Ramesses&people=&lookup-place=e.g.+India%2c+Shanghai%2c+Thebes&place=&from=ad&fromDate=&to=ad&toDate=&lookup-object=e.g.+bowl%2c+hanging+scroll%2c+print&object=&lookup-subject=e.g.+farming%2c+New+Testament&subject=&lookup-matcult=e.g.+Choson+Dynasty%2c+Ptolemaic&matcult=&lookup-technique=e.g.+carved%2c+celadon-glazed&technique=&lookup-school=e.g.+French%2c+Mughal+Style&school=&lookup-material=e.g.+canvas%2c+porcelain%2c+silk&material=&lookup-ethname=e.g.+Hmong%2c+Maori%2c+Tai&ethname=&lookup-ware=e.g.+Imari+ware%2c+Qingbai+ware&ware=&lookup-escape=e.g.+cylinder%2c+gravity%2c+lever&escape=&lookup-bibliography=&bibliography=&citation=&museumno=&ca

In [15]:
cur.executemany("""INSERT INTO britishMuseum (associatedNames, authority, bibliography, culture,
                        curatorComment, dates, denomination, description,
                        materials, museumNumber, objectType, obverseLegend,
                        mint, reverseLegend, state, subjects, weight,
                        url, source, startDate, endDate) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_bm.to_records(index=False)))

In [16]:
cur.execute('''CREATE TABLE americanNumismaticSociety(authority TEXT, deity TEXT, denomination TEXT, description TEXT, 
                       issuer TEXT, mint TEXT, obverseLegend TEXT, portrait TEXT, reference TEXT, reverseLegend TEXT, 
                       url TEXT, weight INT, dates TEXT, source TEXT, startDate INT, endDate INT)''')

In [53]:
cur.executemany("""INSERT INTO americanNumismaticSociety (authority, deity, denomination, description, 
                       issuer, mint, obverseLegend, portrait, reference, reverseLegend, url, weight, dates, 
                       source, startDate, endDate) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", 
                list(cleaned_ans.to_records(index=False)))

In [54]:
cnx.commit()